# <b>Image Metadata ETL

In [17]:
import os
from PIL import Image
import sqlite3
import time

## <b>Extract Data

In [18]:
def extract_image_files(directory):
    image_files = []
    for file in os.listdir(directory):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
            image_files.append(os.path.join(directory, file))
    return image_files

## <b>Transform Data

In [19]:
def transform_image_metadata(image_files):
    metadata = []
    for image_file in image_files:
        with Image.open(image_file) as img:
            resolution = img.size
            file_size = os.path.getsize(image_file)
            image_format = img.format
            mode = img.mode
            last_modified_time = time.ctime(os.path.getmtime(image_file))
            metadata.append({
                'file_name': os.path.basename(image_file),
                'resolution': resolution,
                'file_size': file_size,
                'format': image_format,
                'mode': mode,
                'last_modified': last_modified_time
            })
    return metadata

## <b>Load Data

In [20]:
def load_metadata_to_db(metadata, db_file='image_metadata.db'):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    
    # Drop the existing table if it exists
    c.execute('''
        DROP TABLE IF EXISTS image_metadata
    ''')
    
    # Create a new table with the updated schema
    c.execute('''
        CREATE TABLE IF NOT EXISTS image_metadata (
            file_name TEXT,
            resolution TEXT,
            file_size INTEGER,
            format TEXT,
            mode TEXT,
            last_modified TEXT
        )
    ''')
    
    for data in metadata:
        c.execute('''
            INSERT INTO image_metadata (file_name, resolution, file_size, format, mode, last_modified)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (data['file_name'], f"{data['resolution'][0]}x{data['resolution'][1]}", data['file_size'], data['format'], data['mode'], data['last_modified']))
    
    conn.commit()
    conn.close()

## <b>Main Code Run

In [21]:
if __name__ == "__main__":
    directory = 'Image_Data'
    image_files = extract_image_files(directory)
    metadata = transform_image_metadata(image_files)
    load_metadata_to_db(metadata)